## Copy results 
### FROM: OnTrack export xlsx 
### TO: BoE result xlsx.

!!! Although the code is tested, it is adviced to manually cross check the transferred results. !!!

Author: Ahsan Habib, School of IT, Deakin.

In [10]:
!pip install openpyxl

import openpyxl

In [11]:
def lookup_src(student_id, src_sheet, id_col=3, val_col=4, data_start_row=2):
    row_num = data_start_row
    while True:
        cell_obj = src_sheet.cell(row=row_num, column=id_col)
        # print(cell_obj, cell_obj.value)
        if cell_obj is None or cell_obj.value is None:
            # print("lookup_src cell value is None")
            return
        if int(cell_obj.value) == student_id:
            # print(f"Found ID:{student_id} at row {row_num}")
            # get the grade
            cell_obj = src_sheet.cell(row=row_num, column=val_col)            
            # return round(float(cell_obj.value), 0)
            return cell_obj.value
        row_num += 1


def find_col_by_name(sheet, header_row_num, header_name):
    col = 1
    while True:
        cell_obj = sheet.cell(row=header_row_num, column=col)
        if cell_obj.value == header_name:
            return col
        col += 1

In [14]:
DEST_DATA_START = 13
DEST_ID_COL = 3
DEST_MARK_COL = 7
DEST_XLSX_FILE = "results/SIT225-2024-T2.xlsx"
DEST_WORKBOOK = "Results"

SRC_XLSX_FILE = "results/SIT225-Students-ontrack-all.xlsx"
SRC_WORKBOOK = "SIT225-Students-ontrack-all"
SRC_ID_COL = 3
SRC_MARK_COL = 4
SRC_COMMENT_COL = 5

dest_wb = openpyxl.load_workbook(DEST_XLSX_FILE)
try:
    dest_sheet_obj = dest_wb[DEST_WORKBOOK]
except:
    raise Exception(f"Destination ({DEST_XLSX_FILE}) does not have workbook '{DEST_WORKBOOK}'")

src_wb = openpyxl.load_workbook(SRC_XLSX_FILE)
try:
    src_sheet_obj = src_wb[SRC_WORKBOOK]
except:
    raise Exception(f"Source ({SRC_XLSX_FILE}) does not have workbook '{SRC_WORKBOOK}'")

dest_col_notes = find_col_by_name(dest_sheet_obj, DEST_DATA_START-1, "Notes")
if dest_col_notes is None:
    print(f"Notes column not found in destination header row {DEST_DATA_START-1}")
else:
    print("Notes:", dest_col_notes)

dst_update_count = 0
dst_row = DEST_DATA_START
dst_missing_ids = []
while True:
    cell_obj = dest_sheet_obj.cell(row=dst_row, column=DEST_ID_COL)
    if cell_obj is None or cell_obj.value is None:
        break
    dst_id = int(cell_obj.value)
    # print(dst_id)    

    found = lookup_src(dst_id, src_sheet_obj, id_col=SRC_ID_COL, val_col=SRC_MARK_COL)
    if found is not None:
        dest_sheet_obj.cell(row=dst_row, column=DEST_MARK_COL).value = round(float(found), 0)
        dst_update_count += 1

        # Find comments
        found = lookup_src(dst_id, src_sheet_obj, id_col=SRC_ID_COL, val_col=SRC_COMMENT_COL)
        if found is not None:
            dest_sheet_obj.cell(row=dst_row, column=dest_col_notes).value = found
    else:
        dst_missing_ids.append(dst_id)
    dst_row += 1

dst_total_count = dst_row - DEST_DATA_START
print(f"Total {dst_update_count} rows updated out of {dst_total_count}, missing:{dst_total_count-dst_update_count}.")
print(f"Missing IDs: {dst_missing_ids}")
dest_wb.save("results/SIT225-2024-T2-updated.xlsx")

Notes: 96
Total 94 rows updated out of 112, missing:18.
Missing IDs: [223246624, 224324435, 223045108, 219123969, 222524354, 224455121, 224470288, 224453808, 222398231, 223330914, 224108926, 223171213, 224415265, 219355343, 217090531, 100033634, 220570299, 224600566]
